In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests
import re
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

In [2]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

#opciones.add_argument('--incognito')              # incognito

PATH= './chromedriver'
driver = webdriver.Chrome(PATH, options=opciones)

In [3]:
driver.get('https://www.atrapalo.com/hoteles')

In [4]:
driver.find_element_by_class_name("default-input.tt-input").click()


In [5]:
driver.find_element_by_class_name("default-input.tt-input").send_keys("menorca")

In [6]:
driver.find_element_by_class_name("boton-buscar.form-submit.columns.large-1.large-uncentered.show-for-large").click()

In [7]:
time.sleep(10)
driver.find_element_by_xpath('//*[@id="sortingContainer"]/nav/div[2]/ul/li[2]').click()

In [8]:
texto_columnas = driver.find_element_by_class_name('hotels-list.box-resultados')

## Sacar Hoteles

In [9]:
texto_hoteles = driver.find_elements_by_class_name('openFontSemiBold')
texto_hoteles = texto_hoteles[:4]

In [10]:
time.sleep(10)
nombre = []
for i in texto_hoteles:
    nombre.append(i.text)
    time.sleep(5)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=90.0.4430.212)


## Sacar Precio

In [ ]:
time.sleep(6)
texto_precio = texto_columnas.find_elements_by_class_name('value')
texto_precio = texto_precio[0:4]

In [ ]:
precio = []
for i in texto_precio:
    precio.append(i.text)
    time.sleep(1)

## Sacar dirección

In [ ]:
hotel = driver.find_elements_by_class_name('product-name')

In [ ]:
direcc = []
for i in hotel:
    primero = i.get_attribute("href")
    direcc.append(primero)

In [ ]:
direcc = direcc[0:4]

In [ ]:
driver.quit()

In [ ]:
time.sleep(5)
address = []
for link in direcc:
    driver = webdriver.Chrome("./chromedriver")
    driver.get(link)
    time.sleep(3)
    calle = driver.find_element_by_css_selector("span.app-address")
    calle = calle.text
    address.append(calle)
    driver.quit()

In [ ]:
nombre

In [ ]:
precio

In [ ]:
address

In [ ]:
df = pd.DataFrame(list(zip(nombre,precio,address)), columns = ['Nombre','Precio','Direccion'])
df